# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f3670d8b5b0>
├── 'a' --> tensor([[-0.2713,  0.9071, -1.5831],
│                   [-0.9901,  0.3521, -0.0310]])
└── 'x' --> <FastTreeValue 0x7f3670d163a0>
    └── 'c' --> tensor([[ 0.2481, -1.2715,  0.1181, -0.3858],
                        [-0.1869, -0.6934, -0.5251, -0.9666],
                        [ 1.3810,  0.9269, -0.6675,  0.4139]])

In [4]:
t.a

tensor([[-0.2713,  0.9071, -1.5831],
        [-0.9901,  0.3521, -0.0310]])

In [5]:
%timeit t.a

63.8 ns ± 0.0472 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f3670d8b5b0>
├── 'a' --> tensor([[ 0.4456, -0.0099,  0.8542],
│                   [ 0.2772,  1.4803,  0.0907]])
└── 'x' --> <FastTreeValue 0x7f3670d163a0>
    └── 'c' --> tensor([[ 0.2481, -1.2715,  0.1181, -0.3858],
                        [-0.1869, -0.6934, -0.5251, -0.9666],
                        [ 1.3810,  0.9269, -0.6675,  0.4139]])

In [7]:
%timeit t.a = new_value

67.2 ns ± 0.0836 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.2713,  0.9071, -1.5831],
               [-0.9901,  0.3521, -0.0310]]),
    x: Batch(
           c: tensor([[ 0.2481, -1.2715,  0.1181, -0.3858],
                      [-0.1869, -0.6934, -0.5251, -0.9666],
                      [ 1.3810,  0.9269, -0.6675,  0.4139]]),
       ),
)

In [10]:
b.a

tensor([[-0.2713,  0.9071, -1.5831],
        [-0.9901,  0.3521, -0.0310]])

In [11]:
%timeit b.a

55 ns ± 0.0252 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.7732, -0.6196,  1.1367],
               [-0.6762, -0.0310,  0.9926]]),
    x: Batch(
           c: tensor([[ 0.2481, -1.2715,  0.1181, -0.3858],
                      [-0.1869, -0.6934, -0.5251, -0.9666],
                      [ 1.3810,  0.9269, -0.6675,  0.4139]]),
       ),
)

In [13]:
%timeit b.a = new_value

491 ns ± 0.119 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

858 ns ± 1.95 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 29.3 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

139 µs ± 405 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

143 µs ± 988 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f3648180790>
├── 'a' --> tensor([[[-0.2713,  0.9071, -1.5831],
│                    [-0.9901,  0.3521, -0.0310]],
│           
│                   [[-0.2713,  0.9071, -1.5831],
│                    [-0.9901,  0.3521, -0.0310]],
│           
│                   [[-0.2713,  0.9071, -1.5831],
│                    [-0.9901,  0.3521, -0.0310]],
│           
│                   [[-0.2713,  0.9071, -1.5831],
│                    [-0.9901,  0.3521, -0.0310]],
│           
│                   [[-0.2713,  0.9071, -1.5831],
│                    [-0.9901,  0.3521, -0.0310]],
│           
│                   [[-0.2713,  0.9071, -1.5831],
│                    [-0.9901,  0.3521, -0.0310]],
│           
│                   [[-0.2713,  0.9071, -1.5831],
│                    [-0.9901,  0.3521, -0.0310]],
│           
│                   [[-0.2713,  0.9071, -1.5831],
│                    [-0.9901,  0.3521, -0.0310]]])
└── 'x' --> <FastTreeValue 0x7f3670d16cd0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.1 µs ± 45.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f364818b310>
├── 'a' --> tensor([[-0.2713,  0.9071, -1.5831],
│                   [-0.9901,  0.3521, -0.0310],
│                   [-0.2713,  0.9071, -1.5831],
│                   [-0.9901,  0.3521, -0.0310],
│                   [-0.2713,  0.9071, -1.5831],
│                   [-0.9901,  0.3521, -0.0310],
│                   [-0.2713,  0.9071, -1.5831],
│                   [-0.9901,  0.3521, -0.0310],
│                   [-0.2713,  0.9071, -1.5831],
│                   [-0.9901,  0.3521, -0.0310],
│                   [-0.2713,  0.9071, -1.5831],
│                   [-0.9901,  0.3521, -0.0310],
│                   [-0.2713,  0.9071, -1.5831],
│                   [-0.9901,  0.3521, -0.0310],
│                   [-0.2713,  0.9071, -1.5831],
│                   [-0.9901,  0.3521, -0.0310]])
└── 'x' --> <FastTreeValue 0x7f3670d16370>
    └── 'c' --> tensor([[ 0.2481, -1.2715,  0.1181, -0.3858],
                        [-0.1869, -0.6934, -0.5251, -0.9666],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 58.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59 µs ± 141 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.2481, -1.2715,  0.1181, -0.3858],
                       [-0.1869, -0.6934, -0.5251, -0.9666],
                       [ 1.3810,  0.9269, -0.6675,  0.4139]],
              
                      [[ 0.2481, -1.2715,  0.1181, -0.3858],
                       [-0.1869, -0.6934, -0.5251, -0.9666],
                       [ 1.3810,  0.9269, -0.6675,  0.4139]],
              
                      [[ 0.2481, -1.2715,  0.1181, -0.3858],
                       [-0.1869, -0.6934, -0.5251, -0.9666],
                       [ 1.3810,  0.9269, -0.6675,  0.4139]],
              
                      [[ 0.2481, -1.2715,  0.1181, -0.3858],
                       [-0.1869, -0.6934, -0.5251, -0.9666],
                       [ 1.3810,  0.9269, -0.6675,  0.4139]],
              
                      [[ 0.2481, -1.2715,  0.1181, -0.3858],
                       [-0.1869, -0.6934, -0.5251, -0.9666],
                       [ 1.3810,  0.9269, -0.6675,  0.4139]],

In [26]:
%timeit Batch.stack(batches)

78.4 µs ± 220 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.2481, -1.2715,  0.1181, -0.3858],
                      [-0.1869, -0.6934, -0.5251, -0.9666],
                      [ 1.3810,  0.9269, -0.6675,  0.4139],
                      [ 0.2481, -1.2715,  0.1181, -0.3858],
                      [-0.1869, -0.6934, -0.5251, -0.9666],
                      [ 1.3810,  0.9269, -0.6675,  0.4139],
                      [ 0.2481, -1.2715,  0.1181, -0.3858],
                      [-0.1869, -0.6934, -0.5251, -0.9666],
                      [ 1.3810,  0.9269, -0.6675,  0.4139],
                      [ 0.2481, -1.2715,  0.1181, -0.3858],
                      [-0.1869, -0.6934, -0.5251, -0.9666],
                      [ 1.3810,  0.9269, -0.6675,  0.4139],
                      [ 0.2481, -1.2715,  0.1181, -0.3858],
                      [-0.1869, -0.6934, -0.5251, -0.9666],
                      [ 1.3810,  0.9269, -0.6675,  0.4139],
                      [ 0.2481, -1.2715,  0.1181, -0.3858],
                   

In [28]:
%timeit Batch.cat(batches)

143 µs ± 222 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

316 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
